In [2]:
import spacy
from spacy.language import Language
from spacy.pipeline import EntityRuler

## Ici, nous avons deux versions du calcul complexité et sa corrélation entre le niveau du recette et la complexité.
### La première version se présente dans les deux première cellules et la deuxième version se présente dans les deux dernière cellules.
### Les différences entre deux versions sont les fichiers de la liste d'ingrédients et de la liste de verbes. 
### Dans la première version, nous avons utilisé le fichier IngredientParSpacy.txt et verbesCusineSurInternet.txt pour les entités nommées "INGREDIENT" et "VERBE". Dans la deuxième version, nous avons utilisé le fichier listIngredientsToSave.csv et verbesSpacyCorrect.txt pour les entités nommées  "INGREDIENT" et "VERBE".

In [3]:
"""
Consignes AjoutEntiteNommee
Entrée : trois fichiers txt des listes de mots pour trois types d'entités nommées : INGREDIENT, VERBE(PROCESSUS), USTENSILE
Sortie : ajouter les entités nommées dans le pipeline du spacy "fr_core_news_sm"
"""
ingredients = []
with open ("Data/IngredientParSpacy.txt", "r") as file:

    for item in file:
        item = item.split("\n")[0]
        ingredients.append(item)
verbes = []
with open("Data/verbesCusineSurInternet.txt", "r") as file2:
    for item in file2:
        item = item.split("(")[0].split("\n")[0]
        verbes.append(item)
ustensiles = []
with open("Data/ustensilesCuisineSurInternet.txt", "r") as file3:
    for item in file3:
        item = item.split("\n")[0]
        ustensiles.append(item)

nlp = spacy.load("fr_core_news_sm")

@Language.factory('rulerV30')
def RulerV21(nlp, name):

    ingredient_pattern = [
        {"LOWER": {"IN": ingredients}}
    ]
    verbe_pattern = [
        {"LOWER": {"IN": verbes}}
    ]
    ustensile_pattern = [
        {"LOWER": {"IN": ustensiles}}
    ]
    patterns = [{"label": "INGREDIENT", "pattern": ingredient_pattern}, {"label": "VERBE", "pattern": verbe_pattern}, {"label": "USTENSILE", "pattern": ustensile_pattern}]
    return EntityRuler(nlp, patterns=patterns)

nlp.add_pipe('rulerV30', before="ner")

In [5]:

#calcul de la complexité en temps
"""
Consigne:
Pré-requis : Trois entités nommées sont déjà ajouté dans le pipeline, donc ici, on a déjà nlp
Entrée : le dossier qui contient les fichiers xml de recettes culinaires
Sortie : Un dictionnaire, sa clé est le niveau de recette mentionnée dans le fichier xml, son valeur est la multiplication de la complexité en temps
"""
import xml.dom.minidom as xmldom
import glob
import re
import spacy

for file in glob.glob('2014/*.xml'):
    xml_file = xmldom.parse(file)
    eles = xml_file.documentElement
    #on récupere les éléments d'ingrédients
    ingre = eles.getElementsByTagName("p")
    ingredients = []
    recettes = []
    for i in range(len(ingre)):
        ingredient = ingre[i].firstChild.data
        ingredients.append(ingredient) #la liste d'ingrédient par recette
    n_ingres = 0 #compteur pour compter tous les ingrédients de chaque recette
    textes = [] #les ingrédient
    num=[] #la quantité pour chaque ingrédient
    for ingredient in ingredients:
        #la préparation pour le nombre d'opération
        ingredient = re.sub(" et ", " + ", ingredient) #on remplace et à + pour faciliter le split après
        if ingredient[0].isnumeric(): #si les ingrédient contiennt le numéro, on peut savoir les quantité, sinon, c'est les ingrédient tels que sel, poivre, on traite comme une unité
            ingredient_ent = nlp(ingredient)
            for ent in ingredient_ent.ents:
                if ent.label_ == 'INGREDIENT':
                    textes.append(ent.text) 
            if int(ingredient[0]) < 10: #si la quantité est trop gros, ça veut dire il utilise la quantité comme g, cl etc, du coup on traite comme une unité  
                num.append(int(ingredient[0]))
            else:
                num.append(1)
        else:
            pattern = re.compile(r"[,|+] ") 
            ingredient_2 = pattern.split(ingredient) #on découpe les phrases d'ingrédient par , et + pour récupérer tous les ingrédients
            for ingre in ingredient_2:
                num.append(1)
                if ' ' in ingre: #s'il n'y pas d'espace, ça veut dire que c'est un seul mot tels que sel, nous pouvons ajouter dans la liste text directement. Sinon, il faut reconnaître les ingredient 
                    ingre = nlp(ingre)
                    for ent in ingre.ents:
                        if ent.label_ == 'INGREDIENT': #on cherche les ingrédients dans la phrase d'ingredint
                            textes.append(ent.text)
                else:
                    textes.append(ingre)

        match = re.search(r"g|kg|cl|grams|kilograms|lb|lbs|pounds|cs|cc|litre|litres", ingredient)
        #pour les ingrédients qui sont dans la liste de search, on traite comme une unité, pour les autres, on ajoute les nombres comme il présente dans la recette
        if match:
            n_ingres += 1        
        else:
            find_num = re.findall("\d+\/?\d*",ingredient)
            if find_num:
                for item in find_num:
                    if "/" in item: #si le nombre est écrit comme le format 1/2, on va transformer en 0.5
                        item1 = int(item.split("/")[0])
                        item2 = int(item.split('/')[1])
                        res = item1/item2
                        n_ingres += res
                    else:
                        n_ingres += int(item)
        if n_ingres < 1 :
            n_ingres == 1

    #Normalment, les quatres lignes suivantes n'existent pas, mais notre programme n'est pas parfait, il peut pas identifier tous les ingredient parfaitement, du coup, la longeur de la liste num et de la liste textes ne sont pas identifiques, pour créer un dic, on traite comme ça
    if len(num) > len(textes):
        num = num[0:len(textes)]
    if len(num) < len(textes):
        textes = textes[0:len(num)]
    #à partir d'ici, la longueur de deux listes sont pareils, on peut créer un dictionnaire
    dict_num_ingre = dict(zip(textes, num)) #la clé est l'ingrédients, le valeur est son quantité
    #print(dict_num_ingre)
    
    #l'opération et le calcul en opération
    text = eles.getElementsByTagName("preparation")[0].firstChild.wholeText
    text = re.sub("\n", "", text)
    text = re.sub("   ", "", text)
    pattern = re.compile(r"[,.;] ")
    texts = pattern.split(text) #on découpe les textes en ",.;"
    phrases = []
    for phrase in texts:
        phrase = phrase.lower()
        if phrase == "":
            continue
        phrases.append(phrase)
    phrase_ingre = []
    phrase_verbe = []
    list_espace = []
    list_temps = []
    list_niveau = []
    partie_operation1 = 0
    partie_operation2 = 0
    for phrase in phrases:
        nlp_phrase = nlp(phrase)
        #On obtient deux listes, une liste qui a le label Ingredient, une autre qui a le label Verbe
        #Et après, avec ces deux listes, on obtient la liste des phrases qu'elles ont que les labels verbe et on calcule le nombre de phrase
        for ent in nlp_phrase.ents:
            if ent.label_ == "INGREDIENT":
                phrase_ingre.append(phrase)                
            elif ent.label_ == "VERBE":
                phrase_verbe.append(phrase)
    verbe_seul = len(set(phrase_verbe)^set(phrase_ingre))#On obtient le nombre de la phrase qui n'a pas d'entité nommée INGREDIENT
    ingredient_seul = list(set(phrase_ingre)^set(phrase_verbe)) #on obteint le nombre de la phrase qui n'a pas d'entité nommée VERBE
    for ingredient in ingredient_seul:
        for key, value in dict_num_ingre.items():
            if key in ingredient:
                partie_operation1 += value #on récupére les quantités d'ingrédient, ils ont utlisé au moins une fois dans la recette, donc, on ajoute dans le nombre d'opération
    #ingredient_seul_length = len(set(phrase_ingre)^set(phrase_verbe))
    intersection_verbe_ingre = list(set(phrase_ingre) & (set(phrase_verbe))) 
    for ingredient in intersection_verbe_ingre:
        for key, value in dict_num_ingre.items():
            if key in ingredient:
                partie_operation2 += value #si VERBE et INGREDIENT sont tous dans la phrase, on calcule le nombre d'opération normalement
    #inter_verbe_ingre_length = len(set(phrase_ingre) & (set(phrase_verbe)))
    res_operation = verbe_seul + partie_operation1 + partie_operation2 #resultat du nombre d'opération dans une recette
    res_temps = (res_operation/n_ingres)*n_ingres * 1 #resultat de la complexité en temps d'une recette
    list_temps.append(res_temps)
    ele_niveau = eles.getElementsByTagName("niveau") #on récupere l'élément niveau
    for i in range(len(ele_niveau)):
        niveau = ele_niveau[i].firstChild.data
    list_niveau.append(niveau)
    dict_correlation = dict(zip(list_niveau,list_temps)) #la clé est le niveau du recette, le valeur est la complexité en temps

    print(dict_correlation) #le résultat

{'Facile': 26.0}
{'Facile': 39.0}
{'Facile': 18.0}
{'Très facile': 17.0}
{'Facile': 33.0}
{'Très facile': 26.0}
{'Facile': 78.0}
{'Facile': 17.0}
{'Facile': 9.0}
{'Moyennement difficile': 22.0}
{'Très facile': 12.0}
{'Facile': 13.0}
{'Très facile': 19.0}
{'Facile': 52.0}
{'Très facile': 17.0}
{'Très facile': 29.000000000000004}
{'Facile': 25.0}
{'Facile': 20.0}
{'Très facile': 12.0}
{'Facile': 20.0}
{'Moyennement difficile': 56.0}
{'Très facile': 11.0}
{'Très facile': 18.0}
{'Facile': 13.0}
{'Facile': 13.0}
{'Facile': 13.0}
{'Très facile': 8.0}
{'Facile': 28.0}
{'Facile': 33.0}
{'Très facile': 14.0}
{'Très facile': 25.0}
{'Très facile': 21.0}
{'Très facile': 13.0}
{'Facile': 16.0}
{'Très facile': 22.0}
{'Facile': 32.0}
{'Facile': 59.99999999999999}
{'Facile': 11.0}
{'Facile': 38.0}
{'Facile': 18.0}
{'Facile': 11.0}
{'Très facile': 45.0}
{'Très facile': 47.0}
{'Très facile': 31.0}
{'Facile': 21.0}
{'Facile': 36.0}
{'Facile': 48.0}
{'Moyennement difficile': 30.0}
{'Très facile': 3.0}
{'F

{'Facile': 14.000000000000002}
{'Très facile': 19.0}
{'Très facile': 17.0}
{'Facile': 19.0}
{'Très facile': 34.0}
{'Très facile': 47.0}
{'Très facile': 3.0}
{'Très facile': 17.0}
{'Très facile': 23.0}
{'Facile': 11.0}
{'Facile': 22.0}
{'Facile': 50.0}
{'Facile': 23.0}
{'Très facile': 12.0}
{'Facile': 27.0}
{'Facile': 35.0}
{'Très facile': 12.0}
{'Facile': 36.0}
{'Très facile': 23.0}
{'Très facile': 18.0}
{'Facile': 23.0}
{'Facile': 31.0}
{'Facile': 17.0}
{'Moyennement difficile': 5.0}
{'Très facile': 30.0}
{'Très facile': 39.0}
{'Très facile': 18.0}
{'Très facile': 15.0}
{'Facile': 26.0}
{'Très facile': 11.0}
{'Facile': 21.000000000000004}
{'Très facile': 17.0}
{'Facile': 6.0}
{'Très facile': 15.0}
{'Facile': 58.0}
{'Facile': 32.0}
{'Moyennement difficile': 45.0}
{'Facile': 10.0}
{'Très facile': 25.0}
{'Très facile': 21.0}
{'Facile': 16.0}
{'Moyennement difficile': 27.0}
{'Facile': 71.0}
{'Moyennement difficile': 48.0}
{'Très facile': 9.0}
{'Très facile': 19.0}
{'Facile': 16.0}
{'Très 

{'Facile': 17.0}
{'Très facile': 23.0}
{'Très facile': 23.0}
{'Facile': 30.0}
{'Très facile': 9.0}
{'Très facile': 18.0}
{'Très facile': 18.0}
{'Facile': 23.0}
{'Facile': 33.0}
{'Facile': 54.0}
{'Moyennement difficile': 36.0}
{'Moyennement difficile': 35.0}
{'Très facile': 21.0}
{'Facile': 82.0}
{'Facile': 32.0}
{'Moyennement difficile': 111.0}
{'Moyennement difficile': 84.0}
{'Très facile': 18.0}
{'Facile': 8.0}
{'Très facile': 10.0}
{'Facile': 20.0}
{'Moyennement difficile': 14.0}
{'Facile': 47.0}
{'Très facile': 11.0}
{'Facile': 16.0}
{'Facile': 91.00000000000001}
{'Très facile': 22.0}
{'Moyennement difficile': 28.0}
{'Très facile': 16.0}
{'Très facile': 36.0}
{'Facile': 30.0}
{'Facile': 28.0}
{'Facile': 50.0}
{'Très facile': 17.0}
{'Très facile': 22.0}
{'Facile': 31.0}
{'Facile': 18.0}
{'Très facile': 12.0}
{'Très facile': 21.000000000000004}
{'Très facile': 14.0}
{'Très facile': 4.0}
{'Facile': 15.0}
{'Très facile': 66.0}
{'Très facile': 27.0}
{'Très facile': 20.0}
{'Facile': 57.0

ZeroDivisionError: division by zero

In [20]:
ingredients = []
with open ("Data/listIngredientsToSave.csv", "r") as file:
    for item in file:
        item = item.split("\n")[0]
        ingredients.append(item)
verbes = []
with open("Data/verbesSpacyCorrect.txt", "r") as file2:
    for item in file2:
        item = item.split("\n")[0].split(" ")[0]
        verbes.append(item)
ustensiles = []
with open("ustensilesCuisine.txt", "r") as file3:
    for item in file3:
        item = item.split("\n")[0]
        ustensiles.append(item)

nlp = spacy.load("fr_core_news_sm")

@Language.factory('rulerV33')
def RulerV32(nlp, name):

    ingredient_pattern = [
        {"LOWER": {"IN": ingredients}}
    ]
    verbe_pattern = [
        {"LOWER": {"IN": verbes}}
    ]
    ustensile_pattern = [
        {"LOWER": {"IN": ustensiles}}
    ]
    patterns = [{"label": "INGREDIENT", "pattern": ingredient_pattern}, {"label": "VERBE", "pattern": verbe_pattern}, {"label": "USTENSILE", "pattern": ustensile_pattern}]
    return EntityRuler(nlp, patterns=patterns)

nlp.add_pipe('rulerV33', before="ner")

In [26]:

#calcul de la complexité en temps
"""
Consigne:
Pré-requis : Trois entités nommées sont déjà ajouté dans le pipeline, donc ici, on a déjà nlp
Entrée : le dossier qui contient les fichiers xml de recettes culinaires
Sortie : Un dictionnaire, sa clé est le niveau de recette mentionnée dans le fichier xml, son valeur est la multiplication de la complexité en temps
"""
import xml.dom.minidom as xmldom
import glob
import re
import spacy

for file in glob.glob('2014/*.xml'):
    xml_file = xmldom.parse(file)
    eles = xml_file.documentElement
    #on récupere les éléments d'ingrédients
    ingre = eles.getElementsByTagName("p")
    ingredients = []
    recettes = []
    for i in range(len(ingre)):
        ingredient = ingre[i].firstChild.data
        ingredients.append(ingredient) #la liste d'ingrédient par recette
    n_ingres = 0 #compteur pour compter tous les ingrédients de chaque recette
    textes = [] #les ingrédient
    num=[] #la quantité pour chaque ingrédient
    for ingredient in ingredients:
        #la préparation pour le nombre d'opération
        ingredient = re.sub(" et ", " + ", ingredient) #on remplace et à + pour faciliter le split après
        if ingredient[0].isnumeric(): #si les ingrédient contiennt le numéro, on peut savoir les quantité, sinon, c'est les ingrédient tels que sel, poivre, on traite comme une unité
            ingredient_ent = nlp(ingredient)
            for ent in ingredient_ent.ents:
                if ent.label_ == 'INGREDIENT':
                    textes.append(ent.text) 
            if int(ingredient[0]) < 10: #si la quantité est trop gros, ça veut dire il utilise la quantité comme g, cl etc, du coup on traite comme une unité  
                num.append(int(ingredient[0]))
            else:
                num.append(1)
        else:
            pattern = re.compile(r"[,|+] ") 
            ingredient_2 = pattern.split(ingredient) #on découpe les phrases d'ingrédient par , et + pour récupérer tous les ingrédients
            for ingre in ingredient_2:
                num.append(1)
                if ' ' in ingre: #s'il n'y pas d'espace, ça veut dire que c'est un seul mot tels que sel, nous pouvons ajouter dans la liste text directement. Sinon, il faut reconnaître les ingredient 
                    ingre = nlp(ingre)
                    for ent in ingre.ents:
                        if ent.label_ == 'INGREDIENT': #on cherche les ingrédients dans la phrase d'ingredint
                            textes.append(ent.text)
                else:
                    textes.append(ingre)

        match = re.search(r"g|kg|cl|grams|kilograms|lb|lbs|pounds|cs|cc|litre|litres", ingredient)
        #pour les ingrédients qui sont dans la liste de search, on traite comme une unité, pour les autres, on ajoute les nombres comme il présente dans la recette
        if match:
            n_ingres += 1        
        else:
            find_num = re.findall("\d+\/?\d*",ingredient)
            if find_num:
                for item in find_num:
                    if "/" in item: #si le nombre est écrit comme le format 1/2, on va transformer en 0.5
                        item1 = int(item.split("/")[0])
                        item2 = int(item.split('/')[1])
                        res = item1/item2
                        n_ingres += res
                    else:
                        n_ingres += int(item)
        if n_ingres < 1 :
            n_ingres == 1

    #Normalment, les quatres lignes suivantes n'existent pas, mais notre programme n'est pas parfait, il peut pas identifier tous les ingredient parfaitement, du coup, la longeur de la liste num et de la liste textes ne sont pas identifiques, pour créer un dic, on traite comme ça
    if len(num) > len(textes):
        num = num[0:len(textes)]
    if len(num) < len(textes):
        textes = textes[0:len(num)]
    #à partir d'ici, la longueur de deux listes sont pareils, on peut créer un dictionnaire
    dict_num_ingre = dict(zip(textes, num)) #la clé est l'ingrédients, le valeur est son quantité
    #print(dict_num_ingre)
    
    #l'opération et le calcul en opération
    text = eles.getElementsByTagName("preparation")[0].firstChild.wholeText
    text = re.sub("\n", "", text)
    text = re.sub("   ", "", text)
    pattern = re.compile(r"[,.;] ")
    texts = pattern.split(text) #on découpe les textes en ",.;"
    phrases = []
    for phrase in texts:
        phrase = phrase.lower()
        if phrase == "":
            continue
        phrases.append(phrase)
    phrase_ingre = []
    phrase_verbe = []
    list_espace = []
    list_temps = []
    list_niveau = []
    partie_operation1 = 0
    partie_operation2 = 0
    for phrase in phrases:
        nlp_phrase = nlp(phrase)
        #On obtient deux listes, une liste qui a le label Ingredient, une autre qui a le label Verbe
        #Et après, avec ces deux listes, on obtient la liste des phrases qu'elles ont que les labels verbe et on calcule le nombre de phrase
        for ent in nlp_phrase.ents:
            if ent.label_ == "INGREDIENT":
                phrase_ingre.append(phrase)                
            elif ent.label_ == "VERBE":
                phrase_verbe.append(phrase)
    verbe_seul = len(set(phrase_verbe)^set(phrase_ingre))#On obtient le nombre de la phrase qui n'a pas d'entité nommée INGREDIENT
    ingredient_seul = list(set(phrase_ingre)^set(phrase_verbe)) #on obteint le nombre de la phrase qui n'a pas d'entité nommée VERBE
    for ingredient in ingredient_seul:
        for key, value in dict_num_ingre.items():
            if key in ingredient:
                partie_operation1 += value #on récupére les quantités d'ingrédient, ils ont utlisé au moins une fois dans la recette, donc, on ajoute dans le nombre d'opération
    #ingredient_seul_length = len(set(phrase_ingre)^set(phrase_verbe))
    intersection_verbe_ingre = list(set(phrase_ingre) & (set(phrase_verbe))) 
    for ingredient in intersection_verbe_ingre:
        for key, value in dict_num_ingre.items():
            if key in ingredient:
                partie_operation2 += value #si VERBE et INGREDIENT sont tous dans la phrase, on calcule le nombre d'opération normalement
    #inter_verbe_ingre_length = len(set(phrase_ingre) & (set(phrase_verbe)))
    res_operation = verbe_seul + partie_operation1 + partie_operation2 #resultat du nombre d'opération dans une recette
    res_temps = (res_operation/n_ingres)*n_ingres * 1 #resultat de la complexité en temps d'une recette
    list_temps.append(res_temps)
    ele_niveau = eles.getElementsByTagName("niveau") #on récupere l'élément niveau
    for i in range(len(ele_niveau)):
        niveau = ele_niveau[i].firstChild.data
    list_niveau.append(niveau)
    dict_correlation = dict(zip(list_niveau,list_temps)) #la clé est le niveau du recette, le valeur est la complexité en temps

    print(dict_correlation) #le résultat

{'Facile': 17.0}
{'Facile': 22.0}
{'Facile': 15.0}
{'Très facile': 9.0}
{'Facile': 27.0}
{'Très facile': 10.0}
{'Facile': 30.0}
{'Facile': 23.0}
{'Facile': 11.0}
{'Moyennement difficile': 18.0}
{'Très facile': 9.0}
{'Facile': 12.0}
{'Très facile': 6.0}
{'Facile': 18.0}
{'Très facile': 25.0}
{'Très facile': 10.0}
{'Facile': 12.0}
{'Facile': 8.0}
{'Très facile': 0.0}
{'Facile': 7.0}
{'Moyennement difficile': 40.0}
{'Très facile': 1.0}
{'Très facile': 3.0}
{'Facile': 2.0}
{'Facile': 4.0}
{'Facile': 9.0}
{'Très facile': 11.0}
{'Facile': 8.0}
{'Facile': 10.0}
{'Très facile': 6.0}
{'Très facile': 13.0}
{'Très facile': 20.0}
{'Très facile': 9.0}
{'Facile': 10.0}
{'Très facile': 14.0}
{'Facile': 0.0}
{'Facile': 18.0}
{'Facile': 12.0}
{'Facile': 6.0}
{'Facile': 15.0}
{'Facile': 7.0}
{'Très facile': 11.0}
{'Très facile': 21.0}
{'Très facile': 36.0}
{'Facile': 9.0}
{'Facile': 16.0}
{'Facile': 6.0}
{'Moyennement difficile': 18.0}
{'Très facile': 1.0}
{'Facile': 4.0}
{'Très facile': 10.0}
{'Très fa

{'Très facile': 9.0}
{'Très facile': 1.0}
{'Très facile': 8.0}
{'Très facile': 17.0}
{'Facile': 6.0}
{'Facile': 5.0}
{'Facile': 16.0}
{'Facile': 3.0}
{'Très facile': 0.0}
{'Facile': 24.0}
{'Facile': 13.0}
{'Très facile': 3.0}
{'Facile': 19.0}
{'Très facile': 18.0}
{'Très facile': 5.0}
{'Facile': 10.0}
{'Facile': 5.0}
{'Facile': 6.0}
{'Moyennement difficile': 0.0}
{'Très facile': 16.0}
{'Très facile': 10.0}
{'Très facile': 4.0}
{'Très facile': 6.0}
{'Facile': 16.0}
{'Très facile': 11.0}
{'Facile': 15.0}
{'Très facile': 8.0}
{'Facile': 9.0}
{'Très facile': 0.0}
{'Facile': 63.0}
{'Facile': 42.0}
{'Moyennement difficile': 18.0}
{'Facile': 8.0}
{'Très facile': 1.0}
{'Très facile': 8.0}
{'Facile': 14.0}
{'Moyennement difficile': 20.0}
{'Facile': 8.0}
{'Moyennement difficile': 39.0}
{'Très facile': 2.0}
{'Très facile': 41.0}
{'Facile': 12.0}
{'Très facile': 12.0}
{'Facile': 10.0}
{'Facile': 23.0}
{'Très facile': 6.0}
{'Facile': 4.0}
{'Facile': 15.0}
{'Très facile': 33.0}
{'Facile': 16.0}
{'Tr

{'Moyennement difficile': 31.0}
{'Moyennement difficile': 19.0}
{'Très facile': 13.0}
{'Facile': 2.0}
{'Très facile': 3.0}
{'Facile': 10.0}
{'Moyennement difficile': 9.0}
{'Facile': 9.0}
{'Très facile': 8.0}
{'Facile': 12.0}
{'Facile': 58.0}
{'Très facile': 18.0}
{'Moyennement difficile': 18.0}
{'Très facile': 11.0}
{'Très facile': 2.0}
{'Facile': 26.0}
{'Facile': 5.0}
{'Facile': 47.0}
{'Très facile': 21.0}
{'Très facile': 11.0}
{'Facile': 14.0}
{'Facile': 7.0}
{'Très facile': 14.0}
{'Très facile': 6.0}
{'Très facile': 0.0}
{'Très facile': 7.0}
{'Facile': 10.0}
{'Très facile': 20.0}
{'Très facile': 11.0}
{'Très facile': 1.0}
{'Facile': 29.0}
{'Très facile': 18.0}
{'Très facile': 16.0}
{'Facile': 19.0}
{'Très facile': 5.0}
{'Très facile': 12.0}
{'Très facile': 27.0}
{'Très facile': 4.0}
{'Facile': 7.0}
{'Très facile': 29.999999999999996}
{'Très facile': 2.0}
{'Très facile': 11.0}
{'Facile': 15.0}
{'Très facile': 7.0}
{'Moyennement difficile': 10.0}
{'Très facile': 9.0}
{'Très facile': 6

ZeroDivisionError: division by zero